### Notebook for the inference of cell-cell communications on COPD-IAV data using `LIANA+`

- **Developed by**: Carlos Talavera-López
- **Würzburg Institute for Systems Immunology, Faculty of Medicine, Julius-Maximilian-Universität Würzburg**
- **Created**: 231109
- **Latest version**: 231109

### Import required modules

In [6]:
import anndata
import numpy as np
import liana as li
import pandas as pd
import scanpy as sc

from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean

### Set up working environment

In [2]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

-----
anndata     0.10.3
scanpy      1.9.6
-----
PIL                 10.1.0
appnope             0.1.3
asttokens           NA
comm                0.2.0
cycler              0.12.1
cython_runtime      NA
dateutil            2.8.2
debugpy             1.8.0
decorator           5.1.1
exceptiongroup      1.1.3
executing           2.0.1
h5py                3.10.0
ipykernel           6.26.0
ipywidgets          8.1.1
jedi                0.19.1
joblib              1.3.2
kiwisolver          1.4.5
liana               1.0.2
llvmlite            0.41.1
matplotlib          3.8.1
mizani              0.9.3
mpl_toolkits        NA
mudata              0.2.3
natsort             8.4.0
numba               0.58.1
numpy               1.26.1
packaging           23.2
pandas              2.1.1
parso               0.8.3
patsy               0.5.3
pexpect             4.8.0
platformdirs        3.11.0
plotnine            0.12.4
prompt_toolkit      3.0.39
psutil              5.9.6
ptyprocess          0.7.0
pure_eval     

### Read in data

In [3]:
adata_all = sc.read_h5ad('../../../data/Marburg_cell_states_locked_scANVI_ctl230901.raw.h5ad') 
adata_all

/Users/cartalop/mambaforge/envs/liana+/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.


AnnData object with n_obs × n_vars = 97573 × 27208
    obs: 'sex', 'age', 'ethnicity', 'PaCO2', 'donor', 'infection', 'disease', 'SMK', 'illumina_stimunr', 'bd_rhapsody', 'n_genes', 'doublet_scores', 'predicted_doublets', 'batch', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_mt2', 'n_counts', 'percent_chrY', 'XIST-counts', 'S_score', 'G2M_score', 'condition', 'sample_group', 'IAV_score', 'group', 'Viral_score', 'cell_type', 'cell_states', 'leiden', 'cell_compartment', 'seed_labels', '_scvi_batch', '_scvi_labels', 'C_scANVI'
    var: 'mt', 'ribo'
    obsm: 'X_scANVI', 'X_scVI', 'X_umap'

### Check `LIANA+` available methods

In [4]:
li.mt.show_methods()

,Method Name,Magnitude Score,Specificity Score,Reference
0,CellPhoneDB,lr_means,cellphone_pvals,"Efremova, M., Vento-Tormo, M., Teichmann, S.A...."
0,Connectome,expr_prod,scaled_weight,"Raredon, M.S.B., Yang, J., Garritano, J., Wang..."
0,log2FC,None,lr_logfc,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,NATMI,expr_prod,spec_weight,"Hou, R., Denisenko, E., Ong, H.T., Ramilowski,..."
0,SingleCellSignalR,lrscore,None,"Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ..."
0,CellChat,lr_probs,cellchat_pvals,"Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha..."
0,Rank_Aggregate,magnitude_rank,specificity_rank,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,Geometric Mean,lr_gmeans,gmean_pvals,CellPhoneDBv2's permutation approach applied t...


### Run `cellphoneDB` for trial

In [8]:
cellphonedb(adata_all, groupby = 'cell_states', 
            expr_prop = 0.1, 
            resource_name = 'consensus', 
            verbose = True, 
            key_added = 'cpdb_res',
            use_raw = False)
adata_all.uns['cpdb_res'].head()

Using `.X`!
/Users/cartalop/mambaforge/envs/liana+/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
Make sure that normalized counts are passed!
/Users/cartalop/mambaforge/envs/liana+/lib/python3.10/site-packages/liana/method/_pipe_utils/_pre.py:144: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
['NC_026431.1', 'NC_026432.1', 'NC_026433.1', 'NC_026434.1', 'NC_026435.1', 'NC_026436.1', 'NC_026437.1', 'NC_026438.1'] contain `_`. Consider replacing those!
Using resource `consensus`.
0.08 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 97573 samples and 27208 features


100%|██████████| 1000/1000 [03:03<00:00,  5.46it/s]


,ligand,ligand_complex,ligand_means,ligand_props,receptor,receptor_complex,receptor_means,receptor_props,source,target,lr_means,cellphone_pvals
987236,SLPI,SLPI,1278.597168,1.0,PLSCR1,PLSCR1,35.724625,1.000000,mixed_Goblet2,ifn_Goblet,657.160889,0.0
93082,SLPI,SLPI,1278.597168,1.0,PLSCR1,PLSCR1,27.771564,0.998572,mixed_Goblet2,SERPINE1+Basal,653.184387,0.0
785864,SLPI,SLPI,1278.597168,1.0,PLSCR1,PLSCR1,26.151073,0.974820,mixed_Goblet2,KRT16+SupraB,652.374146,0.0
428293,SLPI,SLPI,1278.597168,1.0,PLSCR1,PLSCR1,21.818146,0.999192,mixed_Goblet2,MHCII+Club,650.207642,0.0
184745,SLPI,SLPI,1278.597168,1.0,PLSCR1,PLSCR1,21.258554,0.997415,mixed_Goblet2,DHRS9+Club,649.927856,0.0


In [ ]:
adata_all.obs['cell_states'].cat.categories

### Visualise `cellphoneDB` interactions using a dotplot

In [ ]:
li.pl.dotplot(adata = adata_all,
              colour = 'lr_means',
              size = 'cellphone_pvals',
              inverse_size = True, # we inverse sign since we want small p-values to have large sizes
              source_labels = ['SERPINE1+Basal', 'SERPINE2+Basal', 'SERPINE1+Basal'],
              target_labels = ['CD34+', 'CD56+ NK'],
              figure_size = (8, 7),
              # finally, since cpdbv2 suggests using a filter to FPs
              # we filter the pvals column to <= 0.05
              filterby = 'cellphone_pvals',
              filter_lambda = lambda x: x <= 0.05,
              uns_key = 'cpdb_res'
             )

### Run `cellchat` for trial

In [9]:
cellchat(adata_all, groupby = 'cell_states', 
            expr_prop = 0.1, 
            resource_name = 'consensus', 
            verbose = True, 
            key_added = 'ccdb_res',
            use_raw = False)
adata_all.uns['ccdb_res'].head()

Using `.X`!
/Users/cartalop/mambaforge/envs/liana+/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
Make sure that normalized counts are passed!
/Users/cartalop/mambaforge/envs/liana+/lib/python3.10/site-packages/liana/method/_pipe_utils/_pre.py:144: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
['NC_026431.1', 'NC_026432.1', 'NC_026433.1', 'NC_026434.1', 'NC_026435.1', 'NC_026436.1', 'NC_026437.1', 'NC_026438.1'] contain `_`. Consider replacing those!
Using resource `consensus`.
0.08 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 97573 samples and 27208 features


 11%|█▏        | 113/1000 [04:45<38:17,  2.59s/it]